1. Naïve Bayes

In [72]:
# Classifiers
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Sklearn tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import re
import spacy
from itertools import chain
import numpy as np

from pprint import pprint as pp



# Pattern to detect urls
r = """(?:(?:https?|ftp):\/\/|\b(?:[a-z\d]+\.))(?:(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\([^\s()<>]+\)))?\))+(?:\((?:[^\s()<>]+|(?:\(?:[^\s()<>]+\)))?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))?"""


class my_nbayes():
    
    def __init__(self):
        self.weights = {}
        
    
    
    def fit(self, X_train, Y_train):
        """Train classifier"""
        
        # Find dimensions
        n = len(X_train)     # for the training set
        m = len(X_train)[0]  # for each vector in training set
        
        self.array = np.zeros()
        
        # Traverse the training set
        for i in range(0, n):
            # Traverse each vector
            for j in range(0, m):
                # Calculate probability of each label for each feature
        
        
        return self
    
    
    def predict(self, X_predict):
        pass


def read_data(n):
    
    tags = []
    texts = []
    
    f = open('./smsspamcollection/SMSSpamCollection.txt', 'r')
    
    for i in range(n):
        tag, text = f.readline().split('	')
        tags.append(tag)
        texts.append(text)
    
    f.close()
    
    return tags, texts

def normalize(lemma):
    """Some text normalization pre-processors"""
    
    def has_phone_num(lemma):
        if len(lemma) in range(9, 12) and lemma.isdigit():
            return True
        else:
            return False
        
    def is_currency(lemma):
        if lemma in set('$€£'):
            return True
        else:
            return False
        
    def has_url(lemma):
        if len(re.findall(r, lemma)) > 0:
            return True
        else:
            return False
        
    if has_phone_num(lemma):
        return 'PHONE_NUM'
    elif is_currency(lemma):
        return 'CURRENCY'
    elif has_url(lemma):
        return 'URL'
    else:
        return lemma
    

def bag_of_words(text, nlp):
    tokens = nlp(text)
    
    stopwords = set([
        '\n'
    ])
    
    bag = {}
    
    for token in tokens:
        lemma = normalize(token.lemma_)
        
        if lemma not in stopwords:
            if lemma not in bag:
                bag[lemma] = 1
            else:
                bag[lemma] += 1

    return bag


def n_grams(tokens, n=1):
    """Returns an iterator over the n-grams given a list of tokens"""
    shiftToken = lambda i: (el for j,el in enumerate(tokens) if j>=i)
    shiftedTokens = (shiftToken(i) for i in range(n))
    tupleNGrams = zip(*shiftedTokens)
    return tupleNGrams # if join in generator : (" ".join(i) for i in tupleNGrams)


def range_ngrams(tokens, ngramRange=(1,2)):
    """Returns an itirator over all n-grams for n in range(ngramRange) given a list of tokens."""
    return chain(*(n_grams(tokens, i) for i in range(*ngramRange)))


def bag_of_ngrams(text, nlp, n=2):

    doc = nlp(text)
    
    stopwords = set([
        '\n'
    ])
    
    bag = {}
    tokens = []
    
    # Lemmatize, normalize text
    for token in doc:
        lemma = normalize(token.lemma_)
        if lemma not in stopwords:
            tokens.append(lemma)
        
    for ngram in n_grams(tokens, n):
        if ngram not in bag:
            bag[ngram] = 1
        else:
            bag[ngram] += 1

    return bag


def extract_features(bags):
    """Extract baseline list of features"""
    
    features = set()
    
    for bag in bags:
        for word in bag.keys():
            if word not in features:
                features.add(word)
                
    # Turn features into a list to iterate over it
    features = list(features)
    
    return features

def vectorize(bags, features):
    """Vectorize routine for one text"""    
    X = []

    for bag in bags:
        vector = []
        for feature in features:
            if feature in bag:
                vector.append(bag[feature])
            else:
                vector.append(0)
        
        X.append(vector)
        
    return X
            

def prepare_data(total_size, training_split=0.33):
    tags, texts = read_data(total_size)
    nlp = spacy.load("en_core_web_md")
    bags = [bag_of_words(text, nlp) for text in texts]
    features = extract_features(bags)

    Y = tags
    X = vectorize(bags, features)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    
    return X_train, X_test, Y_train, Y_test, features


def dtree_classify(X_train, X_test, Y_train, Y_test):
    """Run classification with Decision trees"""
    
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    
    print('DTree Classification:')
    print(classification_report(Y_test, Y_pred))
    
    return clf

    
def nbayes_classify(X_train, X_test, Y_train, Y_test):
    clf = GaussianNB()
    clf = clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    
    print('Gaussian Naive Bayes Classification:')
    print(classification_report(Y_test, Y_pred))

    return clf


    
def knn_classify(X_train, X_test, Y_train, Y_test):
    clf = KNeighborsClassifier(n_neighbors=3)
    clf = clf.fit(X_train, Y_train)
    Y_pred = clf.predict(X_test)
    
    print('kNN Classification:')
    print(classification_report(Y_test, Y_pred))

    return clf


X_train, X_test, Y_train, Y_test, features = prepare_data(4000)
clf = dtree_classify(X_train, X_test, Y_train, Y_test)


# nbayes_classify(X_train, X_test, Y_train, Y_test)
# knn_classify(X_train, X_test, Y_train, Y_test)

DTree Classification:
              precision    recall  f1-score   support

         ham       0.98      0.99      0.99      1158
        spam       0.93      0.88      0.90       162

   micro avg       0.98      0.98      0.98      1320
   macro avg       0.96      0.93      0.94      1320
weighted avg       0.98      0.98      0.98      1320



In [79]:
sms = "Win free tickets to cinema Call now: 0667933456"

# nlp = spacy.load("en_core_web_md")
X = vectorize([bag_of_words(sms, nlp)], features)
Y = clf.predict(X)[0]

print(Y)

                   

# print(clf.predict(X)[0])


spam
